In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load and preprocess the data

# Read the data from TSV files
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Encode labels: 0 for 'ham' and 1 for 'spam'
train_labels = train_data['label'].apply(lambda x: 1 if x == 'spam' else 0).values
test_labels = test_data['label'].apply(lambda x: 1 if x == 'spam' else 0).values

# Tokenize the text messages
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data['message'])

# Convert text messages to sequences
train_sequences = tokenizer.texts_to_sequences(train_data['message'])
test_sequences = tokenizer.texts_to_sequences(test_data['message'])

# Pad the sequences to have the same length
max_length = max(len(x) for x in train_sequences)
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Convert to numpy arrays
train_padded = np.array(train_padded)
test_padded = np.array(test_padded)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
# Define the model

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

# Ensure max_length is correctly calculated
max_length = max(len(x) for x in train_sequences)
print("Max length of sequences:", max_length)

# Update the model definition
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 16),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(train_padded, train_labels, epochs=25, validation_data=(test_padded, test_labels), verbose=2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded_sequence = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    prediction = model.predict(padded_sequence)[0]

    label = "spam" if prediction > 0.5 else "ham"
    return [prediction[0], label]


# Test the function with an example
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

loss, accuracy = model.evaluate(test_padded, test_labels, verbose=0)
print(f'Test set accuracy: {accuracy:.4f}')

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
